In [6]:
suppressMessages(library(rwwa))

varnm <- "rx3day"
nsamp <- 1000

rp <- switch(varnm, "rx3day" = 10, "pr-jjaso" = 20)

mdl_args <- list("rx3day" = list(dist = "gev", type = "fixeddisp"),
                 "pr-jjaso" = list(dist = "norm", type = "fixeddisp"))[[varnm]]

## CORDEX

In [ ]:
for (fnm in list.files("ts", pattern = paste0(varnm,"_CAM"), full.names = T)) {

    gcm <- gsub("_", "_rcp85_", gsub("i1p1","",paste0(strsplit(fnm, "_")[[1]][3:4], collapse = "_")))
    mdl <- gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][3:5], collapse = "_"))
    print(mdl)
    # if (mdl %in% c("CNRM-CM5_r1i1p1_ALADIN53","HadGEM2-ES_r1i1p1_HadREM3-GA7-05")) next
                
    res_fnm <- gsub("ts/", "res-mdl-gmstonly/res-cordex_", gsub(".nc", ".csv", fnm))
    if (file.exists(res_fnm)) next
     
    gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/cmip5_gmsts/",
                           pattern = paste0(gcm), full.names = T)
    if(length(gmst_fnm) == 0) next

    # load all the data
    ts <- load_ts(fnm, col.names = c("year", "x"))
    gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
    df <- merge(gmst, ts)
    df <- df[df$year <= 2070,]
    df$x <- as.numeric(df$x)
    df <- df[!is.na(df$x),]
    if (nrow(df) < 75) { next }
    
    # pick up model to be fitted depending on variable
    mdl <- tryCatch({
            do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F)))
        }, error = function(cond) {
            return(NULL)
        })
    if (is.null(mdl)) { 
        mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F, "method" = "Nelder-Mead")))
    }
    
    # use model's 2025 GMST to define factual climate        
    cov_now <- df[df$year == 2025,"gmst",drop = F]
    cov_hist <- cov_now - 1.3
    cov_fut <- cov_now + 1.3

    # bootstrap results
    res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                          y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp)
    res <- cbind(res, data.frame("ystart" = min(df$year), "yend" = max(df$year)))
    write.csv(res, res_fnm)

    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res.+/res", "mdl-eval/rlplot", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub("ts/", "", gsub(".csv", "", fnm)))
        }; dev.off()
    }
}
print("Done.")

[1] "CanESM2_r1i1p1_CRCM5"


## HighResMIP

In [ ]:
for(fnm in list.files("ts", pattern = paste0(varnm,"_day"), full.names = T)) {
    
    gcm <- gsub("_highresSST","",paste0(strsplit(fnm, "_")[[1]][3:5], collapse = "_"))
    
    res_fnm <- paste0("res-mdl-gmstonly/res-highresmip_",varnm,"_",gcm,".csv")
    
    if(!file.exists(res_fnm)) {
        ts <- read.table(fnm, col.names = c("year", "x"))

        gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/highresmip/GSAT", pattern = gcm, full.names = T)
        if(length(gmst_fnm) == 0) next
        
        gmst <- load_ts(gmst_fnm, col.names = c("year", "gmst"))
        df <- merge(ts, gmst)
        df$x <- as.numeric(df$x)
        df <- df[!is.na(df$x),]
        if (nrow(df) < 75) { next }
        
        # pick up model to be fitted depending on variable
        mdl <- tryCatch({
                do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F)))
            }, error = function(cond) {
                return(NULL)
            })
        if (is.null(mdl)) { 
            mdl <- do.call(fit_ns, append(mdl_args, list("data" = df, "varnm" = "x", covnm = "gmst", lower = F, "method" = "Nelder-Mead")))
        }
        
        # use model's 2025 GMST to define factual climate        
        cov_now <- df[df$year == 2025,"gmst",drop = F]
        cov_hist <- cov_now - 1.3
        cov_fut <- cov_now + 1.3
    
        # bootstrap results
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                              y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp)
        res <- cbind(res, data.frame("ystart" = min(df$year), "yend" = max(df$year)))
        write.csv(res, res_fnm)

        # quick return level plots to check fit
        rlplot_fnm <- gsub("csv", "png", gsub("res.+/res", "mdl-eval/rlplot", res_fnm))
        if(!file.exists(rlplot_fnm)) {
            png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
                plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub("ts/", "", gsub(".csv", "", fnm)))
            }; dev.off()
        }
    }
}

## Compile models

In [ ]:
for (framing %in% c("cordex", "highresmip")) {
    mdl_res <- t(sapply(list.files("res-mdl-gmstonly", pattern = paste0(framing,".+",varnm), full.names = T), read.csv))
    rownames(mdl_res) <- apply(sapply(strsplit(gsub(".csv","",rownames(mdl_res)), "_"), "[", 3:6), 2, paste0, collapse = "_")
    write.csv(mdl_res, paste0("res-",framing,"_",varnm,"_gmst-only.csv"))
}